In [0]:
if not any(mount.mountPoint == "/mnt/azuremountcfgc" for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
        source = "wasbs://gccontainer@cfgcstorage.blob.core.windows.net",
        mount_point = "/mnt/azuremountcfgc",
        extra_configs = {"fs.azure.account.key.cfgcstorage.blob.core.windows.net":"yyHYfFxpI0AWxsOdFJH699cGYQh60CutcVcsrJfo5UMPixDJdcRTNSudo+Nfz9/lrMZTxyAjNiZO+AStaG7M1w=="}
)

In [0]:
from pyspark.sql.functions import col, max as max_

In [0]:
def get_dir_content(ls_path):
    dir_paths = dbutils.fs.ls(ls_path)
    subdir_paths = [get_dir_content(p.path) for p in dir_paths if p.isDir() and p.path != ls_path]
    flat_subdir_paths = [p for subdir in subdir_paths for p in subdir]
    return list(map(lambda p: p.path, dir_paths)) + flat_subdir_paths

In [0]:
paths = [p for p in get_dir_content("/mnt/azuremountcfgc/output") if '.parquet' in p]
paths

Out[4]: ['dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00000-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-2-1.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00000-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-10-1.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00001-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-3-1.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00001-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-11-1.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00000-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-2-2.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00000-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-10-2.c000.snappy.parquet',
 'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/pa

In [0]:
sorted_files = {}
for path in paths:
    path_l = path.split('/')
    date = path_l[4].split('_')[0]
    if date in sorted_files:
        if path_l[5] in sorted_files[date]:
            sorted_files[date][path_l[5]].append(path)
        else:
            sorted_files[date][path_l[5]] = [path]
    else:
         sorted_files[date] = {path_l[5]: [path]}

In [0]:
sorted_files

Out[6]: {'2020-08-05': {'partition=Q': ['dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00000-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-2-1.c000.snappy.parquet',
   'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00000-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-10-1.c000.snappy.parquet',
   'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00001-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-3-1.c000.snappy.parquet',
   'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00001-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-11-1.c000.snappy.parquet'],
  'partition=T': ['dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00000-tid-2324541136500767637-0aac7cda-d548-469a-9eab-01898f16ab48-2-2.c000.snappy.parquet',
   'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00000-tid-6951345584299579629-ae63999a-0c7b-4079-9b51-efe29a796f6e-10-2.c000.snappy.parquet',
  

In [0]:
merged_list = list(zip(trade_paths, quote_paths))

Out[8]: [('dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00000-tid-8769688038772410899-c57386a3-921f-4b8d-b8c6-336b68f6b7bb-7-2.c000.snappy.parquet',
  'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00000-tid-8769688038772410899-c57386a3-921f-4b8d-b8c6-336b68f6b7bb-7-1.c000.snappy.parquet'),
 ('dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=T/part-00001-tid-8769688038772410899-c57386a3-921f-4b8d-b8c6-336b68f6b7bb-8-2.c000.snappy.parquet',
  'dbfs:/mnt/azuremountcfgc/output/2020-08-05/partition=Q/part-00001-tid-8769688038772410899-c57386a3-921f-4b8d-b8c6-336b68f6b7bb-8-1.c000.snappy.parquet'),
 ('dbfs:/mnt/azuremountcfgc/output/2020-08-06/partition=T/part-00000-tid-292241090050411007-78e0263f-b77e-48b2-9601-d346f4f9f555-10-2.c000.snappy.parquet',
  'dbfs:/mnt/azuremountcfgc/output/2020-08-06/partition=Q/part-00000-tid-292241090050411007-78e0263f-b77e-48b2-9601-d346f4f9f555-10-1.c000.snappy.parquet'),
 ('dbfs:/mnt/azuremountcfgc/output/2020-08-06/partiti

In [0]:
for files in sorted_files:
    trade_common = spark.read.parquet(*sorted_files[files]['partition=T'])
    quote_common = spark.read.parquet(*sorted_files[files]['partition=Q'])
    trade = trade_common.select('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'arrival_tm', 'trade_pr', 'trade_size')
    quote = quote_common.select('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'arrival_tm', 'bid_pr', 'bid_size', 'ask_pr', 'ask_size')
    trade_corrected = trade.groupBy('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'trade_pr').agg(max_('arrival_tm'))
    quote_corrected = quote.groupBy('trade_dt', 'symbol', 'exchange', 'event_tm', 'event_seq_nb', 'bid_pr', 'bid_size', 'ask_pr', 'ask_size').agg(max_('arrival_tm'))
    trade_date = trade_corrected.first()['trade_dt']
    quote_date = quote_corrected.first()['trade_dt']
    trade_corrected.write.mode("overwrite").parquet(f"/mnt/azuremountcfgc/trade/trade_dt={trade_date}")
    quote_corrected.write.mode("overwrite").parquet(f"/mnt/azuremountcfgc/quote/quote_dt={trade_date}")